# 2 Layer Neural Network

## Imports

In [1]:
import os
import pandas as pd

## Import data and create .csv

In [2]:
# Directory to MRI Images
base_dir = '/Users/benrandoing/Downloads/archive/AugmentedAlzheimerDataset'
classes = ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']

if not os.path.exists('dataset.csv'):
    data = []

    for label, class_name in enumerate(classes):
        class_dir = os.path.join(base_dir, class_name)
        for image_name in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image_name)
            data.append([image_path, label])

    df = pd.DataFrame(data, columns=['path', 'label'])
    df.to_csv('dataset.csv', index=False)

## Data Loading and Pre-Processing

In [3]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, random_split
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.dataframe = pd.read_csv(csv_file)
        self.dataframe = self.dataframe.sample(frac=1).reset_index(drop=True)  # Shuffle the dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = self.dataframe.iloc[idx, 0]
        image = Image.open(img_name)
        label = int(self.dataframe.iloc[idx, 1])

        if self.transform:
            image = self.transform(image)

        return image, label

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize image to 224 X 224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    #transforms.Lambda(lambda x: x.view(-1))  # Flatten the image
])

# Load the entire dataset
dataset = CustomDataset(csv_file='dataset.csv', transform=transform)

# Split the dataset into train, validation, and test sets
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

In [4]:
print(f'Training Samples: {len(train_dataset)}')
print(f'Validation Samples: {len(val_dataset)}')
print(f'Test Samples: {len(test_dataset)}')

Training Samples: 23788
Validation Samples: 5097
Test Samples: 5099


In [5]:
print(train_dataset)

In [6]:
import matplotlib.pyplot as plt
#plt.plot(train_dataset[0][0])
#plt.show()

## Pytorch Logistic Regression Model

In [7]:
class FiveLayerNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, hidden_dim3, hidden_dim4, output_dim):
        super(FiveLayerNN, self).__init__()
        self.fc1 = torch.nn.Linear(input_dim, hidden_dim1)
        self.fc2 = torch.nn.Linear(hidden_dim1, hidden_dim2)
        self.fc3 = torch.nn.Linear(hidden_dim2, hidden_dim3)
        self.fc4 = torch.nn.Linear(hidden_dim3, hidden_dim4)
        self.fc5 = torch.nn.Linear(hidden_dim4, output_dim)

    def forward(self, x):
        #x = x.view(-1, 224 * 224 * 3)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.fc5(x)
        return x

input_dim = 224 * 224 * 3  # Assuming the images are RGB
hidden_dim1 = 512
hidden_dim2 = 256
hidden_dim3 = 128
hidden_dim4 = 64
output_dim = 4  # Four classes

#model = FiveLayerNN(input_dim, hidden_dim1, hidden_dim2, hidden_dim3, hidden_dim4, output_dim)
import torchvision
from torchvision.models import resnet50, ResNet50_Weights
model = torchvision.models.resnet50(weights=ResNet50_Weights.DEFAULT)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

model.fc = torch.nn.Linear(model.fc.in_features, output_dim)


## Training Loop

In [8]:
learning_rate = 0.001
num_epochs = 10
batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define the loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    print("num_epochs",num_epochs)
    model.train()
    total_loss = 0.0

    i = 0
    for inputs, labels in train_loader:
        i = i + 1
        print("Training continues..", total_loss)    
        optimizer.zero_grad()
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)        
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Iteration {epoch} - Loss = {total_loss / len(train_loader)}")

num_epochs 10
Training continues.. 0.0
Training continues.. 1.39179527759552
Training continues.. 2.7634741067886353
Training continues.. 4.161164879798889
Training continues.. 5.552160143852234
Training continues.. 6.945839047431946
Training continues.. 8.328164339065552
Training continues.. 9.694830417633057
Training continues.. 11.084020018577576
Training continues.. 12.476722955703735
Training continues.. 13.855401158332825
Training continues.. 15.23708164691925
Training continues.. 16.610545992851257
Training continues.. 18.001039505004883
Training continues.. 19.363448977470398
Training continues.. 20.73610496520996
Training continues.. 22.118954181671143
Training continues.. 23.49728775024414
Training continues.. 24.860013127326965
Training continues.. 26.223979353904724
Training continues.. 27.60044574737549
Training continues.. 28.986473083496094
Training continues.. 30.38008677959442
Training continues.. 31.753417372703552
Training continues.. 33.13677370548248
Training conti

Training continues.. 282.20392632484436
Training continues.. 283.4907270669937
Training continues.. 284.83664774894714
Training continues.. 286.15580117702484
Training continues.. 287.47946429252625
Training continues.. 288.8013769388199
Training continues.. 290.12725734710693
Training continues.. 291.47459387779236
Training continues.. 292.86149299144745
Training continues.. 294.1653525829315
Training continues.. 295.47999107837677
Training continues.. 296.81776678562164
Training continues.. 298.11642611026764
Training continues.. 299.4553390741348
Training continues.. 300.7825617790222
Training continues.. 302.1477243900299
Training continues.. 303.43756222724915
Training continues.. 304.83589267730713
Training continues.. 306.1586674451828
Training continues.. 307.47112119197845
Training continues.. 308.80536675453186
Training continues.. 310.16433811187744
Training continues.. 311.5079424381256
Training continues.. 312.84272265434265
Training continues.. 314.1872401237488
Training 

Training continues.. 56.76515543460846
Training continues.. 58.05515515804291
Training continues.. 59.35505926609039
Training continues.. 60.64578378200531
Training continues.. 61.972100377082825
Training continues.. 63.23225128650665
Training continues.. 64.54157972335815
Training continues.. 65.81189942359924
Training continues.. 67.1120754480362
Training continues.. 68.42688369750977
Training continues.. 69.71103942394257
Training continues.. 70.98601996898651
Training continues.. 72.34635579586029
Training continues.. 73.64721822738647
Training continues.. 74.92491579055786
Training continues.. 76.21528208255768
Training continues.. 77.49572956562042
Training continues.. 78.74843907356262
Training continues.. 80.03935265541077
Training continues.. 81.32510387897491
Training continues.. 82.60049390792847
Training continues.. 83.87053370475769
Training continues.. 85.16542541980743
Training continues.. 86.52510070800781
Training continues.. 87.79523730278015
Training continues.. 89.0

Training continues.. 320.93543887138367
Training continues.. 322.15508258342743
Training continues.. 323.43891620635986
Training continues.. 324.64507830142975
Training continues.. 325.87449383735657
Training continues.. 327.0283272266388
Training continues.. 328.26526045799255
Training continues.. 329.5821237564087
Training continues.. 330.8502058982849
Training continues.. 332.07978546619415
Training continues.. 333.367787361145
Training continues.. 334.6226954460144
Training continues.. 335.8775397539139
Training continues.. 337.0538330078125
Training continues.. 338.2537395954132
Training continues.. 339.50958359241486
Training continues.. 340.75576078891754
Training continues.. 341.990168094635
Training continues.. 343.22761476039886
Training continues.. 344.4445798397064
Training continues.. 345.6432285308838
Training continues.. 346.8759481906891
Training continues.. 348.16700851917267
Training continues.. 349.4191081523895
Training continues.. 350.64881241321564
Training contin

Training continues.. 106.481720328331
Training continues.. 107.7087037563324
Training continues.. 108.86678993701935
Training continues.. 110.05635786056519
Training continues.. 111.26517188549042
Training continues.. 112.46800410747528
Training continues.. 113.58940410614014
Training continues.. 114.78587591648102
Training continues.. 115.94830787181854
Training continues.. 117.15891563892365
Training continues.. 118.33561670780182
Training continues.. 119.55191695690155
Training continues.. 120.78749895095825
Training continues.. 121.97951376438141
Training continues.. 123.20926797389984
Training continues.. 124.42403709888458
Training continues.. 125.58488988876343
Training continues.. 126.72099149227142
Training continues.. 127.87084317207336
Training continues.. 129.0998159646988
Training continues.. 130.34083151817322
Training continues.. 131.50674533843994
Training continues.. 132.72005987167358
Training continues.. 133.91289508342743
Training continues.. 135.1216540336609
Train

...

Training continues.. 72.82518798112869
Training continues.. 73.61663526296616
Training continues.. 74.45014518499374
Training continues.. 75.07209521532059
Training continues.. 75.80178940296173
Training continues.. 76.55029022693634
Training continues.. 77.31138277053833
Training continues.. 78.00308680534363
Training continues.. 78.73074728250504
Training continues.. 79.41312301158905
Training continues.. 80.18479073047638
Training continues.. 80.99468284845352
Training continues.. 81.67225110530853
Training continues.. 82.4544587135315
Training continues.. 83.26959520578384
Training continues.. 84.10364264249802
Training continues.. 84.84519559144974
Training continues.. 85.5923439860344
Training continues.. 86.36108708381653
Training continues.. 87.09912049770355
Training continues.. 87.80205941200256
Training continues.. 88.4722027182579
Training continues.. 89.26976573467255
Training continues.. 89.95503103733063
Training continues.. 90.62427538633347
Training continues.. 91.2978

Training continues.. 225.98141080141068
Training continues.. 226.60671776533127
Training continues.. 227.40264415740967
Training continues.. 228.18811333179474
Training continues.. 229.02597868442535
Training continues.. 229.73226934671402
Training continues.. 230.45686811208725
Training continues.. 231.14116144180298
Training continues.. 231.8389059305191
Training continues.. 232.645887196064
Training continues.. 233.46111196279526
Training continues.. 234.20452058315277
Training continues.. 234.8898265361786
Training continues.. 235.5327004790306
Training continues.. 236.25070238113403
Training continues.. 236.9459839463234
Training continues.. 237.681216776371
Training continues.. 238.32384705543518
Training continues.. 239.10807991027832
Training continues.. 239.73629075288773
Training continues.. 240.4682690501213
Training continues.. 241.1603488922119
Training continues.. 241.80255115032196
Training continues.. 242.47528743743896
Training continues.. 243.23447787761688
Training c

Training continues.. 99.53824335336685
Training continues.. 100.18616336584091
Training continues.. 100.94172614812851
Training continues.. 101.57787102460861
Training continues.. 102.2838968038559
Training continues.. 102.97817689180374
Training continues.. 103.55660969018936
Training continues.. 104.25503897666931
Training continues.. 105.08355957269669
Training continues.. 105.68665957450867
Training continues.. 106.30446821451187
Training continues.. 107.00450772047043
Training continues.. 107.6472619175911
Training continues.. 108.31598019599915
Training continues.. 109.1054298877716
Training continues.. 109.88176822662354
Training continues.. 110.43895536661148
Training continues.. 111.10283654928207
Training continues.. 111.78245401382446
Training continues.. 112.38219422101974
Training continues.. 112.90680313110352
Training continues.. 113.63017678260803
Training continues.. 114.35230773687363
Training continues.. 114.99938559532166
Training continues.. 115.68402487039566
Trai

Training continues.. 242.71896415948868
Training continues.. 243.36151283979416
Training continues.. 244.0274839401245
Training continues.. 244.72596764564514
Training continues.. 245.37704157829285
Training continues.. 246.06397008895874
Training continues.. 246.78930032253265
Training continues.. 247.4325807094574
Training continues.. 248.01361364126205
Training continues.. 248.78496938943863
Training continues.. 249.5811357498169
Training continues.. 250.23648530244827
Training continues.. 250.9795714020729
Training continues.. 251.8110669851303
Training continues.. 252.60720145702362
Training continues.. 253.38345271348953
Training continues.. 254.10615307092667
Training continues.. 254.841024518013
Training continues.. 255.4772973060608
Training continues.. 256.1457536816597
Training continues.. 256.7433983683586
Training continues.. 257.4047337770462
Training continues.. 258.0861193537712
Training continues.. 258.75237077474594
Training continues.. 259.35599637031555
Training con

Training continues.. 122.61361521482468
Training continues.. 123.26480662822723
Training continues.. 123.95355492830276
Training continues.. 124.8005518913269
Training continues.. 125.45438528060913
Training continues.. 126.14654964208603
Training continues.. 126.74607610702515
Training continues.. 127.37454336881638
Training continues.. 128.1895226240158
Training continues.. 128.84524303674698
Training continues.. 129.5576193332672
Training continues.. 130.27954894304276
Training continues.. 130.91333132982254
Training continues.. 131.62520533800125
Training continues.. 132.36560726165771
Training continues.. 133.02516382932663
Training continues.. 133.67384213209152
Training continues.. 134.2676783800125
Training continues.. 134.84638607501984
Training continues.. 135.56039142608643
Training continues.. 136.36700826883316
Training continues.. 136.90639436244965
Training continues.. 137.74452775716782
Training continues.. 138.44260996580124
Training continues.. 138.9795259833336
Train

## Evaluation

In [9]:
# Validation loop
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in val_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

validation_accuracy = 100 * correct / total
print(f"Validation Accuracy: {validation_accuracy}%")

Validation Accuracy: 67.45144202472042%


In [10]:
# Testing loop
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy}%")

Test Accuracy: 67.85644243969406%
